# **Signal To Spike Conversion** - Baseline Detection
This notebook presents the **baseline detection** step that needs to be conducted before doing the signal-to-spike conversion.

The signal-to-spike conversion requires the definition of the `up_threshold` and `down_threshold` values that will be used to translate the continuous signal into discrete events. Here, we follow the approach proposed by Indiveri's team to define these thresholds. Which is composed of the following steps:
1. Select a time window of the signal representative of the whole signal.
   - The source work used a 1s window.
   - We will test different window sizes.
2. Store the maximum signal amplitudes of consecutive non-overlapping time sub-windows of a defined size.
3. Take the mean of the lowest quartile as the baseline amplitude (or other metric)
   - Check why we are using the lowest quartile.    

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Let's first load the filtered SEEG signal
This notebook only converts 1 SEEG channel to spikes. Therefore, the `.npy` file must contain a single channel.

### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [2]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"
INPUT_FOLDER = f"../subset/results/{DATASET_FILENAME}"
RESULTS_FOLDER = "baseline_results"

### Load the Ripple, Fast Ripple, and Annotated Events

In [3]:
import numpy as np
import math
from utils.io import preview_np_array

# Load the filtered seeg signal in the ripple band
ripple_seeg_file_name = f"{INPUT_FOLDER}/ripple_band.npy"
ripple_band_seeg = np.load(f"{ripple_seeg_file_name}")

# Remove the extra inner dimension
ripple_band_seeg = np.squeeze(ripple_band_seeg)

preview_np_array(ripple_band_seeg, "Ripple Band SEEG", edge_items=3)

Ripple Band SEEG Shape: (245760,).
Preview: [1.84829940e-04 1.37975809e-03 3.92964380e-03 ... 1.23691538e+00
 7.54000855e-01 1.73992494e-01]


In [4]:
# Load the filtered seeg signal in the fast ripple band
fr_seeg_file_name = f"{INPUT_FOLDER}/fr_band.npy"
fr_band_seeg = np.load(f"{fr_seeg_file_name}")

# Remove the extra inner dimension
fr_band_seeg = np.squeeze(fr_band_seeg)

preview_np_array(fr_band_seeg, "FR Band SEEG", edge_items=3)

FR Band SEEG Shape: (245760,).
Preview: [ 0.00095524  0.00310525 -0.00478473 ... -0.49596476 -0.2753618
  0.28661303]


In [ ]:
# Load the filtered seeg signal in the HFO band
fr_seeg_file_name = f"{INPUT_FOLDER}/fr_band.npy"
fr_band_seeg = np.load(f"{fr_seeg_file_name}")

# Remove the extra inner dimension
fr_band_seeg = np.squeeze(fr_band_seeg)

preview_np_array(fr_band_seeg, "FR Band SEEG", edge_items=3)

In [6]:
# Load the annotated events (For Ripples)
ripple_markers_file_name = f"{INPUT_FOLDER}/markers_ripple_band.npy"
ripple_markers = np.load(f"{ripple_markers_file_name}")

# Remove the extra inner dimension
ripple_markers = np.squeeze(ripple_markers)

preview_np_array(ripple_markers, "ripple_markers", edge_items=3)

ripple_markers Shape: (222,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]


In [7]:
# Load the annotated events (For Fast Ripples)
fr_markers_file_name = f"{INPUT_FOLDER}/markers_fr_band.npy"
fr_markers = np.load(f"{fr_markers_file_name}")

# Remove the extra inner dimension
fr_markers = np.squeeze(fr_markers)

preview_np_array(fr_markers, "fr_markers", edge_items=3)

fr_markers Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]


There are 222 Ripple Events and 199 Fast Ripple Events in the dataset for the whole time. Since we are considering a 60s window, that means about half of the events are present during the window. Thus, we can **expect to have around 111 Ripple Events and 100 Fast Ripple Events** in the window. -> Relevant Event Rate ~= 105 / 60s = 1.75 Events/s

Therefore, there is an event ~ every `1/1.75s` = `0.571s`.

We can use a `sub_window_size` of `0.5/4` = `0.1428s`. This way, about 25% of the sub-windows will contain a relevant event. This is useful to decide the heuristic that will select the baseline threshold. An intuitive one is to select the Q3 (75th percentile) of the maximum amplitudes of the sub-windows.

## 1. Select a time window of the signal representative of the whole signal

In [312]:
import math
from utils.input import SAMPLING_RATE, X_STEP

window_size = 60 * (10**3)  # 60 seconds or 60,000 ms

# Crop the signal to the respective window_size
# find the last index before the window_size
num_samples_window = math.ceil(window_size / X_STEP)

# Crop the signal to the respective window_size
ripple_band_seeg = ripple_band_seeg[:num_samples_window]
fr_band_seeg = fr_band_seeg[:num_samples_window]

preview_np_array(ripple_band_seeg, "Cropped Ripple Band SEEG", edge_items=3)
preview_np_array(fr_band_seeg, "Cropped FR Band SEEG", edge_items=3)

Cropped Ripple Band SEEG Shape: (122880,).
Preview: [ 1.84829940e-04  1.37975809e-03  3.92964380e-03 ... -9.70567864e-02
 -4.53527518e-01 -7.20332108e-01]
Cropped FR Band SEEG Shape: (122880,).
Preview: [ 0.00095524  0.00310525 -0.00478473 ... -0.29380478 -0.22372997
 -0.05078035]


## Store the maximum signal amplitudes of consecutive non-overlapping time sub-windows
For this, we need to define the length of the `sub_windows`.

Since we floor the value, some values in the end of the window will not be considered if it is not a multiple of the sub-window length

In [313]:
# Each sub-window is 0.1s or 100ms OR let's also try 0.46s
sub_window_scale = 14.28/100
sub_window_length = math.floor((num_samples_window / 60) * sub_window_scale)   # Define the sub-window length according to the window size 
print("Sub-Window Length: ", sub_window_length)

Sub-Window Length:  292


### Iterate over the sub-windows and store the maximum/minimum signal amplitude of each sub-window

In [314]:
ripple_max_amplitudes = []
ripple_min_amplitudes = []
fr_max_amplitudes = []
fr_min_amplitudes = []

for start_idx in range(0, num_samples_window - sub_window_length, sub_window_length):
    end_idx = start_idx + sub_window_length
    # Crop the signal to the respective window_size
    ripple_sub_window_seeg = ripple_band_seeg[start_idx:end_idx]
    fr_sub_window_seeg = fr_band_seeg[start_idx:end_idx]

    # Get the maximum signal amplitude in the sub-window
    ripple_max_amp = np.max(ripple_sub_window_seeg)
    fr_max_amp = np.max(fr_sub_window_seeg)
    # Get the minimum signal amplitude in the sub-window
    ripple_min_amp = np.min(ripple_sub_window_seeg)
    fr_min_amp = np.min(fr_sub_window_seeg)

    # Append the maximum amplitudes to the respective lists
    ripple_max_amplitudes.append(ripple_max_amp)
    fr_max_amplitudes.append(fr_max_amp)
    # Append the minimum amplitudes to the respective lists
    ripple_min_amplitudes.append(ripple_min_amp)
    fr_min_amplitudes.append(fr_min_amp)

ripple_max_amplitudes = np.array(ripple_max_amplitudes)
fr_max_amplitudes = np.array(fr_max_amplitudes)
ripple_min_amplitudes = np.array(ripple_min_amplitudes)
fr_min_amplitudes = np.array(fr_min_amplitudes)

preview_np_array(ripple_max_amplitudes, "Ripple Max Amplitudes", edge_items=3)
preview_np_array(ripple_min_amplitudes, "Ripple Min Amplitudes", edge_items=3)
preview_np_array(fr_max_amplitudes, "FR Max Amplitudes", edge_items=3)
preview_np_array(fr_min_amplitudes, "FR Min Amplitudes", edge_items=3)

num_sub_windows = len(ripple_max_amplitudes)

Ripple Max Amplitudes Shape: (420,).
Preview: [2.8749678  2.59958592 2.51322257 ... 3.18531111 3.07960351 2.23911448]
Ripple Min Amplitudes Shape: (420,).
Preview: [-2.76447771 -2.16734483 -2.26768073 ... -2.59355401 -2.48346681
 -1.9709944 ]
FR Max Amplitudes Shape: (420,).
Preview: [1.21234837 1.40058732 1.10543821 ... 1.40628557 7.9667778  1.57403183]
FR Min Amplitudes Shape: (420,).
Preview: [-1.23287038 -1.38711312 -1.17633501 ... -1.27426537 -7.67292681
 -1.56659685]


### Plot the maximum signal amplitudes of the sub-windows as an Histogram

In [315]:
# Plot the maximum amplitudes of the sub-windows in a histogram
from utils.bar_plot import create_histogram  # Import the function to create the figure

# Define the histogram bins
ripple_min_amp = np.min(ripple_max_amplitudes)
ripple_max_amp = np.max(ripple_max_amplitudes)
bins = np.linspace(ripple_min_amp, ripple_max_amp, 100)    

ripple_max_amp_hist = create_histogram(
    title="Ripple Sub-Window Max. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=ripple_max_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [316]:
# Plot the minimum amplitudes of the sub-windows in a histogram
# Define the histogram bins
ripple_min_amp = np.min(ripple_min_amplitudes)
ripple_max_amp = np.max(ripple_min_amplitudes)
bins = np.linspace(ripple_min_amp, ripple_max_amp, 100)    

ripple_min_amp_hist = create_histogram(
    title="Ripple Sub-Window Min. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=ripple_min_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [317]:
# Define the histogram bins
fr_min_amp = np.min(fr_max_amplitudes)
fr_max_amp = np.max(fr_max_amplitudes)
bins = np.linspace(fr_min_amp, fr_max_amp, 100)    

fr_max_amp_hist = create_histogram(
    title="FR Sub-Window Max. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=fr_max_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [318]:
# Define the histogram bins
fr_min_amp = np.min(fr_min_amplitudes)
fr_max_amp = np.max(fr_min_amplitudes)
bins = np.linspace(fr_min_amp, fr_max_amp, 100)    

fr_min_amp_hist = create_histogram(
    title="FR Sub-Window Min. Amplitude Histogram", 
    x_axis_label='Amplitude (uV)', 
    y_axis_label='Count',
    bins=bins,
    x=fr_min_amplitudes,
    is_density=False,
    sizing_mode="stretch_width",
)

In [319]:
import bokeh.plotting as bplt

show_ripple_hist_plot = False
if show_ripple_hist_plot:
    # Show the plots
    bplt.show(ripple_min_amp_hist)
    bplt.reset_output()

    bplt.show(ripple_max_amp_hist)
    # Close the plot
    bplt.reset_output()

In [320]:
show_fr_hist_plot = False
if show_fr_hist_plot:
    # Show the plot
    bplt.show(fr_min_amp_hist)
    bplt.reset_output()

    bplt.show(fr_max_amp_hist)
    bplt.reset_output()

### Plot the maximum signal amplitudes of the sub-windows as a Bar Plot

In [321]:
from utils.bar_plot import create_box_plot

# Create the BoxPlot
ripple_max_amp_boxplot = create_box_plot(
    title="Boxplot of Ripple Sub-Window Max. Amplitudes", 
    box_arrays=[ripple_max_amplitudes],
    y_axis_label='Amplitude (uV)',
    sizing_mode="stretch_width",
)

show_ripple_barplot = False
if show_ripple_barplot:
    # Show the plot
    bplt.show(ripple_max_amp_boxplot)

ripple_quantiles:  [2.11984932 2.58686257 6.17625567]
Ripple IQR:  4.056406349714173


In [322]:
# Create the BoxPlot
fr_max_amp_boxplot = create_box_plot(
    title="Boxplot of Fast Ripple Sub-Window Max. Amplitudes", 
    box_arrays=[fr_max_amplitudes],
    y_axis_label='Amplitude (uV)',
    sizing_mode="stretch_width",
)

show_fr_barplot = False
if show_fr_barplot:
    # Show the plot
    bplt.show(fr_max_amp_boxplot)

# TODO: Not plotting the boxplot for the minimum amplitudes

ripple_quantiles:  [1.18243694 1.3695555  4.98697579]
Ripple IQR:  3.8045388469503667


### Show some statistics of the maximum sub-window amplitudes

In [323]:
mean_max_ripple_amp = np.mean(ripple_max_amplitudes)
std_max_ripple_amp = np.std(ripple_max_amplitudes)
median_max_ripple_amp = np.median(ripple_max_amplitudes)
print("Mean Max. Ripple Amplitude: ", mean_max_ripple_amp)
print("Std Max. Ripple Amplitude: ", std_max_ripple_amp)
print("Median Max. Ripple Amplitude: ", median_max_ripple_amp)

Mean Max. Ripple Amplitude:  5.766439260369298
Std Max. Ripple Amplitude:  6.635212984563909
Median Max. Ripple Amplitude:  2.5868625735566395


In [324]:
mean_max_fr_amp = np.mean(fr_max_amplitudes)
std_max_fr_amp = np.std(fr_max_amplitudes)
median_max_fr_amp = np.median(fr_max_amplitudes)
print("Mean Max. FR Amplitude: ", mean_max_fr_amp)
print("Std Max. FR Amplitude: ", std_max_fr_amp)
print("Median Max. FR Amplitude: ", median_max_fr_amp)

Mean Max. FR Amplitude:  2.783418964328162
Std Max. FR Amplitude:  2.542256271509498
Median Max. FR Amplitude:  1.3695554985362877


### Show some statistics of the minimum sub-window amplitudes

In [325]:
mean_min_ripple_amp = np.mean(ripple_min_amplitudes)
std_min_ripple_amp = np.std(ripple_min_amplitudes)
median_min_ripple_amp = np.median(ripple_min_amplitudes)
print("Mean Min Ripple Amplitude: ", mean_min_ripple_amp)
print("Std Min Ripple Amplitude: ", std_min_ripple_amp)
print("Median Min Ripple Amplitude: ", median_min_ripple_amp)

Mean Min Ripple Amplitude:  -5.812280676907504
Std Min Ripple Amplitude:  6.534815244379154
Median Min Ripple Amplitude:  -2.7110415909491756


In [326]:
mean_min_fr_amp = np.mean(fr_min_amplitudes)
std_min_fr_amp = np.std(fr_min_amplitudes)
median_min_fr_amp = np.median(fr_min_amplitudes)
print("Mean Min. FR Amplitude: ", mean_min_fr_amp)
print("Std Min. FR Amplitude: ", std_min_fr_amp)
print("Median Min. FR Amplitude: ", median_min_fr_amp)

Mean Min. FR Amplitude:  -2.6991545889051887
Std Min. FR Amplitude:  2.4094855944484306
Median Min. FR Amplitude:  -1.3661981189133912


## 3. Take the mean of the lowest quartile as the baseline amplitude.

In [336]:
#Mean of the lowest quartile (Q1 - 25%)
lowest_quartile_ripple = np.percentile(ripple_max_amplitudes, 25)

lowest_quartile_fr = np.percentile(fr_max_amplitudes, 25)
# TODO: For now using the DN_Threshold as the symetric value of the UP_THRESHOLD

### Store the Amplitude Thresholds that will be used to convert the signal into spikes

In [328]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_q1.npy"
q1_output = [lowest_quartile_ripple, lowest_quartile_fr]
print("Q1 Strategy Output:", q1_output)

EXPORT_THRESHOLD_Q1 = True
if EXPORT_THRESHOLD_Q1:
    np.save(out_filename, np.array(q1_output))

Q1 Strategy Output: [2.1198493153469276, 1.1824369440591151]


## 4. Use the mean of the amplitudes array as the baseline amplitude.

In [329]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_mean.npy"

mean_output = [mean_max_ripple_amp, mean_max_fr_amp]
print("Mean Strategy Output: ", mean_output)

EXPORT_THRESHOLD_MEAN = True
if EXPORT_THRESHOLD_MEAN:
    np.save(out_filename, np.array(mean_output))

Mean Strategy Output:  [5.766439260369298, 2.783418964328162]


## 5. Use the median of the amplitudes array as the baseline amplitude.

In [330]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_median.npy"

median_output = [median_max_ripple_amp, median_max_fr_amp]
print("Median Strategy Output: ", median_output)

EXPORT_THRESHOLD_MEDIAN = True
if EXPORT_THRESHOLD_MEDIAN:
    np.save(out_filename, np.array(median_output))

Median Strategy Output:  [2.5868625735566395, 1.3695554985362877]


## 6. Use the Q3 (75 percentile) of the amplitudes array as the baseline amplitude + Scaling Factor

In [331]:
# Mean of the lowest quartile (Q1 - 25%)
highest_quartile_ripple = np.percentile(ripple_max_amplitudes, 75)

highest_quartile_fr = np.percentile(fr_max_amplitudes, 75)
# TODO: For now using the DN_Threshold as the symetric value of the UP_THRESHOLD

In [332]:
# Define scaling factor for the thresholds
ripple_scaling_factor = 0.6
fr_scaling_factor = 0.3

In [333]:
# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_q3.npy"

q3_output = [highest_quartile_ripple * ripple_scaling_factor, highest_quartile_fr * fr_scaling_factor]
print("Q3 Strategy Output: ", q3_output)

EXPORT_THRESHOLD_Q3 = True
if EXPORT_THRESHOLD_Q3:
    np.save(out_filename, np.array(q3_output))

Q3 Strategy Output:  [3.7057533990366602, 1.4960927373028445]


## 7. Use the (60 percentile) of the amplitudes array as the baseline amplitude.

In [334]:
# 60 Percentile
sixty_percentile_ripple = np.percentile(ripple_max_amplitudes, 60)
sixty_percentile_fr = np.percentile(fr_max_amplitudes, 60)

# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_60perc.npy"

sixty_perc_output = [sixty_percentile_ripple, sixty_percentile_fr]
print("60% Strategy Output: ", sixty_perc_output)

EXPORT_THRESHOLD_60PERC = True
if EXPORT_THRESHOLD_60PERC:
    np.save(out_filename, np.array(sixty_perc_output))

60% Strategy Output:  [2.852737606236808, 1.4640442276537315]


## 7. Use the (70 percentile) of the amplitudes array as the baseline amplitude.

In [335]:
# 70 Percentile
seventy_percentile_ripple = np.percentile(ripple_max_amplitudes, 70)
seventy_percentile_fr = np.percentile(fr_max_amplitudes, 70)

# Store the amplitude thresholds
out_filename = f"{RESULTS_FOLDER}/{DATASET_FILENAME}_thresholds_70perc.npy"

seventy_perc_output = [seventy_percentile_ripple, seventy_percentile_fr]
print("70% Strategy Output: ", seventy_perc_output)

EXPORT_THRESHOLD_70PERC = True
if EXPORT_THRESHOLD_70PERC:
    np.save(out_filename, np.array(seventy_perc_output))

70% Strategy Output:  [3.9723963321210065, 1.7057082759027868]
